Import useful librairies

In [1]:
import pandas as pd
import numpy as np
import requests
import csv
import re
import bs4
from bs4 import BeautifulSoup

## Get the name of the festivals in Switzerland

In [2]:
list_of_festivals=[]
# Defining the url, the 3 enables to have the swiss list while 1 and 2 are for respectively
# France and Belgium
url_name='http://www.routedesfestivals.com/liste-des-festivals-pour-suisse-3.html' 
r=requests.get(url_name)
soup=BeautifulSoup(r.content,'html.parser')


In [3]:
for elems in soup.findAll('li',{'class':'elemList'}): #find all li which class is elemList
    for festival in elems.findAll('li'):
        a=festival.findAll('a')
        url_festival=a[0].get('href')
        name=a[1].contents[0]
        number=re.search(r'\d+',url_festival).group()
        if len(a)>2:
            ville=a[2].get('title')
        else:
            ville=''
            
        info_festival={}
        info_festival['name']=name
        info_festival['url']=url_festival
        info_festival['ville']=ville
        info_festival['Identificateur']=number
        list_of_festivals.append(info_festival)


        

In [4]:
FolderName='RouteDesFestivals'
csv_=FolderName+'/'+'list_de_tous_les_festivals'+'.csv'
pd.DataFrame(list_of_festivals,columns=['name','url','ville','Identificateur']).to_csv(csv_, index=False, encoding='utf-8')



## Get the programmation of each festivals

In [5]:
url_parms='http://routedesfestivals.com/'
url_prog='http://routedesfestivals.com/ajax/archives.php?id='
for rows in list_of_festivals:
    try:
        list_of_artists=pd.DataFrame(columns=['name','day','month','year','ville','festival'])
        url_get_year=url_parms+rows['url']
        r=requests.get(url_get_year)
        soup=BeautifulSoup(r.content,'html.parser')
        years=soup.findAll('option')
        c=soup.findAll('li',{'class':'fest'})
        for concert_2017 in soup.findAll('li',{'class':'fest'}):
            date=concert_2017.findAll('b')
            if len(date)>0:
                n_day=date[0].contents[0]
                month=date[1].contents[0]
                month=month.split(u'\xa0')[0]
            for artist in concert_2017.findAll('a',{'class':'artiste'}):
                    if len(artist.contents)>0:
                        name=artist.contents[0]
                        artist={}
                        artist['name']=name
                        artist['day']=n_day
                        artist['month']=month
                        artist['year']='2017'
                        artist['ville']=rows['ville']
                        artist['festival']=rows['name']
                        list_of_artists=list_of_artists.append(artist, ignore_index=True)
                    
        
        for year_raw in years:
            year=year_raw['value']
            url_get_prog=url_prog+rows['Identificateur']+'&year='+year
            r=requests.get(url_get_prog)
            soup=BeautifulSoup(r.content,'html.parser')
            for day in soup.findAll('li'):
                date=day.findAll('b')
                n_day=date[0].contents[0]
                month=date[1].contents[0]
                month=month.split(u'\xa0')[0]
                for artist_ in day.findAll('a',{'class':'artiste'}):
                    if len(artist_.contents)>0:
                        name=artist_.contents[0]
                        artist={}
                        artist['name']=name
                        artist['day']=n_day
                        artist['month']=month
                        artist['year']=year
                        artist['ville']=rows['ville']
                        artist['festival']=rows['name']
                        list_of_artists=list_of_artists.append(artist, ignore_index=True)
                    
                    

    
    except:
        print('Error while parsing for festival:'+rows['name']+'for year'+year)
        import traceback;
        traceback.print_exc()
        time.sleep(20)
        
    csv_=FolderName+'/'+rows['name']+'.csv'
    if len(list_of_artists)>0:
        pd.DataFrame(list_of_artists,columns=list(list_of_artists.columns)).to_csv(csv_, index=False, encoding='utf-8')
        


        
    

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
